In [1]:
from pathlib import Path
from aocd.models import Puzzle
import numpy as np
import pandas as pd
import itertools
import collections

day = int(Path().resolve().stem.replace("day", ""))

puzzle = Puzzle(2023, day)
lines = puzzle.input_data.split("\n")
puzzle.view()

Day 10 is a simple network problem. Each "pipe" only connects to the previous and next pipe, so this a a graph. From the starting position S, which, we just need to find the furthest node from S in either direction.

Parsing the input into the graph isn't that simple except we are told that S only connects to two points as well, so we can just iteratively follow this.

In [ ]:
pipe_meanings = {
    "|": [(0, 1),(0, -1)], # up, down
    "-": [(1, 0),(-1, 0)], # right, left
    "L": [(1, -1),(-1, 1)], # down right, left up
    "J": [(1, 1),(-1, -1)],  # right up, down left
    "7": [(1, -1),(-1, 1)], # right down, up left
    "F": [(1, 1),(-1, -1)],  # up right, left, down
}

directed_pipe_meanings = {
    "|": [(0, 1),(0, -1)], # up, down
    "-": [(1, 0),(-1, 0)], # right, left
    "L": [(1, -1),(-1, 1)], # down right, left up
    "J": [(1, 1),(-1, -1)],  # right up, down left
    "7": [(1, -1),(-1, 1)], # right down, up left
    "F": [(1, 1),(-1, -1)],  # up right, left, down
}

In [5]:
drawing = np.array([val for line in lines for val in line if val!= '\n']).reshape((len(lines),len(lines[0])))


In [7]:
def get_starting_node() -> tuple[int, int]:
    for row, line in enumerate(lines):
        if "S" in line:
            col = line.index("S")
            return row, col
    raise ValueError("No starting node found")

get_starting_node()

'S'

In [10]:
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path
from typing import Literal

import numpy as np
from aocd.models import Puzzle


class Cardinal(Enum):
    up = -1, 0
    down = 1, 0
    left = 0, -1
    right = 0, 1

class Offsets(Enum):
    up = 1, 0
    down = -1, 0
    left = 0, -1
    right = 0, 1
    up_left = 1, -1
    up_right = 1, 1
    down_left = -1, -1
    down_right = -1, 1

SYMBOL = Literal["|", "-", "L", "J", "7", "F"]

pipe_offsets: dict[SYMBOL, dict[Cardinal, Offsets]] = {
    "|": {Cardinal.up: Offsets.up, Cardinal.down: Offsets.down}, # up, down
    "-": {Cardinal.right: Offsets.right, Cardinal.left: Offsets.left}, # right, left
    "L": {Cardinal.left: Offsets.up_left, Cardinal.down: Offsets.down_right}, # left up, down right
    "J": {Cardinal.down: Offsets.down_left, Cardinal.right: Offsets.up_right},  # down left, right up
    "7": {Cardinal.right: Offsets.down_right,Cardinal.up: Offsets.up_left}, # right down, up left
    "F": {Cardinal.up: Offsets.up_right, Cardinal.left: Offsets.down_left},  # up right, left, down
}

pipe_headings: dict[SYMBOL, dict[Cardinal, Cardinal]] = {
    "|": {Cardinal.up: Cardinal.up, Cardinal.down: Cardinal.down},
    "-": {Cardinal.left: Cardinal.left, Cardinal.right: Cardinal.right},
    "L": {Cardinal.down: Cardinal.right, Cardinal.left: Cardinal.up},
    "J": {Cardinal.down: Cardinal.left, Cardinal.right: Cardinal.up},
    "7": {Cardinal.up: Cardinal.left, Cardinal.right: Cardinal.down},
    "F": {Cardinal.up: Cardinal.right, Cardinal.left: Cardinal.down},
}


@dataclass
class Node:
    row: int
    col: int
    symbol: SYMBOL
    heading_map: dict[Cardinal, Cardinal] = field(init=False)

    def __post_init__(self):
        self.heading_map = pipe_headings[self.symbol]

    def traverse(self, incoming_direction: Cardinal) -> Cardinal:
        """Tarverse in a given heading and return the next row,col and heading"""
        # heading = pipe_headings[self.symbol][incoming_direction]
        # offset = pipe_offsets[self.symbol][incoming_direction]
        return self.heading_map[incoming_direction]
        #return offset, heading


def get_starting_node() -> tuple[int, int]:
    for row, line in enumerate(lines):
        if "S" in line:
            col = line.index("S")
            return row, col
    raise ValueError("No starting node found")


def find_starting_pipe(network) -> tuple[Node, Cardinal]:
    """Find the starting pipe"""
    row, col = get_starting_node()
    for direction in Cardinal:
        new_row = row + direction.value[0]
        new_col = col + direction.value[1]
        symbol = network[new_row, new_col]
        if symbol == ".":
            continue
        node = Node(new_row, new_col, network[new_row, new_col])
        try:
            out_direction = node.traverse(direction)
            return node, out_direction
        except KeyError:
            continue
    raise ValueError("No starting pipe found")


# np.array(lines)

drawing = np.array([val for line in lines for val in line if val != "\n"]).reshape(
    (len(lines), len(lines[0]))
)



def find_traversed(drawing):
    current_node, heading = find_starting_pipe(drawing)
    symbol = current_node.symbol
    traversed = 1

    while symbol != "S":
        new_row = current_node.row + heading.value[0]
        new_col = current_node.col + heading.value[1]
        symbol = drawing[new_row, new_col]
        if symbol == "S":
            break
        current_node = Node(new_row, new_col, symbol)
        heading = current_node.traverse(heading)
        traversed += 1

    return traversed // 2 + traversed % 2

#puzzle.answer_a = find_traversed(drawing)

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


def draw_polygon_from_path(drawing) -> Polygon:

    current_node, heading = find_starting_pipe(drawing)
    symbol = current_node.symbol
    traversed = 1
    path = [(current_node.row, current_node.col)]

    while symbol != "S":
        new_row = current_node.row + heading.value[0]
        new_col = current_node.col + heading.value[1]
        symbol = drawing[new_row, new_col]
        path.append((new_row, new_col))
        if symbol == "S":
            break
        current_node = Node(new_row, new_col, symbol)
        heading = current_node.traverse(heading)
        traversed += 1

    return Polygon(np.array(path))

poly = draw_polygon_from_path(drawing)

In [29]:
x,y = np.meshgrid(np.arange(drawing.shape[0]), np.arange(drawing.shape[1]))


points = np.vstack((x.flatten(),y.flatten())).T
points = np.apply_along_axis(Point, 1, points)


np.apply_along_axis(poly.contains, 0, points).sum()



383

In [39]:
from shapely.prepared import prep

prep_poly = prep(poly)
np.apply_along_axis(prep_poly.contains, 0, points).sum()


383